In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import pickle

In [2]:
mnist = fetch_openml('mnist_784', version=1)

In [3]:
print((np.array(mnist.data.loc[42]).reshape(28, 28) > 0).astype(int))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 

In [4]:
X, y = mnist["data"], mnist["target"].astype(np.uint8)
sorted_X, sorted_y = mnist["data"], mnist["target"].astype(np.uint8)
sorted_y = y.sort_values()
sorted_X.reindex(sorted_y.index)

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
34999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42651,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X_train, X_test = X[:56000], X[56000:]
y_train, y_test = y[:56000], y[56000:]
sorted_X_train, sorted_X_test = sorted_X[:56000], sorted_X[56000:]
sorted_y_train, sorted_y_test = sorted_y[:56000], sorted_y[56000:]

In [6]:
print("sorted_y_train: ")
sorted_y_train.unique()

sorted_y_train: 


array([0, 1, 2, 3, 4, 5, 6, 7], dtype=uint8)

In [7]:
print("sorted_y_test: ")
sorted_y_test.unique()

sorted_y_test: 


array([7, 8, 9], dtype=uint8)

In [8]:
y_train_0 = (y_train == 0)
y_test_0 = (y_test == 0)

In [9]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_0)

SGDClassifier(random_state=42)

In [10]:
train = sgd_clf.predict(X_train)
train_counter = 0
test = sgd_clf.predict(X_test)
test_counter = 0
for i in range(len(train)):
    if(train[i]==y_train_0.iloc[i]):
        train_counter += 1
for i in range(len(test)):
    if(test[i]==y_test_0.iloc[i]):
        test_counter += 1
sgd_acc = []
sgd_acc.append(train_counter/len(train))
sgd_acc.append(test_counter/len(test))
# sgd_acc = []
# sgd_acc.append(sgd_clf.score(X_train, y_train_0))
# sgd_acc.append(sgd_clf.score(X_test, y_test_0))
f = open('sgd_acc.pkl', 'wb')
pickle.dump(sgd_acc, f)
f.close()
sgd_acc

[0.9901964285714285, 0.9877857142857143]

In [11]:
score = cross_val_score(sgd_clf, X_train, y_train_0, cv=3, scoring="accuracy", n_jobs=-1)
f = open('sgd_cva.pkl', 'wb')
pickle.dump(score, f)
f.close()
score

array([0.98650024, 0.98950019, 0.98542805])

In [14]:
sgd_m_clf = SGDClassifier(random_state=42,n_jobs=-1)
sgd_m_clf.fit(X_train, y_train)
y_train_pred = cross_val_predict(sgd_m_clf, X_train,
y_train, cv=3, n_jobs=-1)
conf_mx = confusion_matrix(y_train, y_train_pred)
f = open('sgd_cmx.pkl', 'wb')
pickle.dump(conf_mx, f)
f.close()
conf_mx

array([[5297,    0,   53,   24,    9,   25,   53,    2,   56,   11],
       [   3, 6078,   64,   54,    8,   14,    9,   19,   48,   11],
       [  55,   67, 4669,  246,   53,   26,  101,   85,  247,   11],
       [  42,   15,  170, 4884,   14,  152,   31,   65,  308,   47],
       [  10,   31,   62,   31, 4918,   20,   60,   33,  159,  124],
       [  97,   28,   68,  369,  102, 3766,  168,   22,  364,   76],
       [  54,   16,   83,   19,   58,   46, 5203,    3,   54,    1],
       [  20,   20,   70,  105,   72,   11,    6, 5271,   79,  156],
       [  81,  129,  151,  385,   61,  110,   64,   35, 4382,   62],
       [  44,   30,   32,  129,  415,   47,    1,  376,  274, 4211]],
      dtype=int64)